In [2]:
import backtrader as bt
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import logging
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
import time

ModuleNotFoundError: No module named 'backtrader'

In [125]:
def download_yf_data(ticker: str, start: datetime, end: datetime, retries: int = 5) -> pd.DataFrame:
    wait_time = 5
    for attempt in range(retries):
        try:
            if attempt > 0:
                print(f"Retrying download in {wait_time} seconds...")
                time.sleep(wait_time)
                wait_time *= 2
            print(f"Downloading data for {ticker} from Yahoo Finance...")
            df = yf.download(
                ticker,
                start=start,
                end=end,
                progress=False,
                auto_adjust=True
            )
            if df.empty:
                print("No data received.")
                continue
            df.index = pd.to_datetime(df.index)
            print(f"✅ Successfully downloaded {len(df)} rows of data for {ticker}")
            return df
        except Exception as e:
            print(f"❌ Error: {e}")
            if attempt == retries - 1:
                raise
    return pd.DataFrame()  # Empty on failure


def setup_backtrader(cerebro: bt.Cerebro, data: pd.DataFrame, ticker: str):

    data_feed = bt.feeds.PandasData(
        dataname=data,
        datetime=None,
        open='Open',
        high='High',
        low='Low',
        close='Close',
        volume='Volume',
        openinterest=-1,
        timeframe=bt.TimeFrame.Days
    )

    already_exists = any(getattr(d, '_name', None) == ticker for d in cerebro.datas)

    if not already_exists:
        cerebro.adddata(data_feed, name=ticker)
    else:
        print(f"❗{ticker} 데이터 이미 존재함 - 추가 생략")
    
    print("🚀 Running backtrader analysis...")


In [126]:
class DataStructureAnalysisStrategy(bt.Strategy):
    """
    Backtrader 데이터 구조를 분석하는 전략
    실제 데이터가 어떻게 저장되고 접근되는지 확인
    """
    def __init__(self):
        print("=" * 100)
        print("__init__()")
        print("=" * 100)

        print(f"총 가진 종목 개수: {len(self.datas)}")

        for i, data in enumerate(self.datas):
            print(f"데이터 {i+1}: {getattr(data, '_name', 'Unknown')}")
      

    def start(self):
        print("=" * 100)
        print("start()")
        print("=" * 100)
    
        print(f"\n🔍 보유 종목:")
        for data in self.datas:
            symbol = getattr(data, '_name', 'Unknown')
            print(f"\n📊 {symbol}")


    def next(self):
        print("next()")
        print("=" * 100)
        
        for data in self.datas:
            symbol = getattr(data, '_name', 'Unknown')
            dt = data.datetime.datetime(0)  # 현재 데이터 포인트의 날짜 (0번째 인덱스)

            current_datetime = data.datetime.datetime(0)
            current_date = data.datetime.date(0)
        
            print(f"📅 현재 날짜: {current_date}")
            print(f"🕐 현재 시간: {current_datetime}")
        

            print(f"📈 현재 가격 정보:")
            print(f"  시가: ${data.open[0]:.2f}")
            print(f"  고가: ${data.high[0]:.2f}")
            print(f"  저가: ${data.low[0]:.2f}")
            print(f"  종가: ${data.close[0]:.2f}")
            print(f"  거래량: {data.volume[0]:,.0f}")



In [127]:
# === 다운로드 수행 ===
print("Backtrader 데이터 다운로드 시작...")
cerebro = bt.Cerebro()
cerebro.addstrategy(DataStructureAnalysisStrategy)


start_date = datetime(2025, 1, 1)
end_date = datetime(2025, 1, 10)

tickers = ['AAPL']
datas = {}
for ticker in tickers:
    print(f"{ticker}")
    df = download_yf_data(ticker, start=start_date, end=end_date)
    df.columns = df.columns.droplevel(1)
    datas[ticker] = df



Backtrader 데이터 다운로드 시작...
AAPL
✅ Successfully downloaded 5 rows of data for AAPL


In [128]:
# === backtrader 설정 ===
for ticker, df in datas.items():
    if not df.empty:
        setup_backtrader(cerebro, df, ticker)
    else:
        print(f"❗{ticker} 데이터 setup 실패")



🚀 Running backtrader analysis...


In [129]:
# === 수행 시작 ===
cerebro.run()

__init__()
총 가진 종목 개수: 1
데이터 1: AAPL
start()

🔍 보유 종목:

📊 AAPL
next()
📅 현재 날짜: 2025-01-02
🕐 현재 시간: 2025-01-02 00:00:00
📈 현재 가격 정보:
  시가: $248.33
  고가: $248.50
  저가: $241.24
  종가: $243.26
  거래량: 55,740,700
next()
📅 현재 날짜: 2025-01-03
🕐 현재 시간: 2025-01-03 00:00:00
📈 현재 가격 정보:
  시가: $242.77
  고가: $243.59
  저가: $241.31
  종가: $242.77
  거래량: 40,244,100
next()
📅 현재 날짜: 2025-01-06
🕐 현재 시간: 2025-01-06 00:00:00
📈 현재 가격 정보:
  시가: $243.72
  고가: $246.73
  저가: $242.61
  종가: $244.41
  거래량: 45,045,600
next()
📅 현재 날짜: 2025-01-07
🕐 현재 시간: 2025-01-07 00:00:00
📈 현재 가격 정보:
  시가: $242.40
  고가: $244.96
  저가: $240.77
  종가: $241.63
  거래량: 40,856,000
next()
📅 현재 날짜: 2025-01-08
🕐 현재 시간: 2025-01-08 00:00:00
📈 현재 가격 정보:
  시가: $241.34
  고가: $243.12
  저가: $239.47
  종가: $242.12
  거래량: 37,628,900


In [130]:
@dataclass
class ScreeningCriteria:
    """스크리닝 조건"""
    min_price: float = 10.0
    max_price: float = 1000.0
    min_volume: int = 100_000
    min_market_cap: float = 1_000_000_000  # 10억 달러
    sectors: List[str] = None
    bottom_breakout_pct: float = 5.0    # 바닥 돌파 %

In [131]:
class SmartStockScreener:
    """
    스마트 주식 스크리너
    - 단계별 필터링으로 효율성 극대화
    - 필요한 데이터만 선별적 다운로드
    """
    
    def __init__(self):
        self.logger = self._setup_logging()
        
    def _setup_logging(self):
        import logging
        logging.basicConfig(level=logging.INFO)
        return logging.getLogger(__name__)


In [132]:
import os
from typing import List, Dict
class BasicInfoScreener(SmartStockScreener):
    """
    기본 정보만으로 1차 필터링
    - 가격, 거래량, 시가총액으로 대부분 걸러냄
    - 빠르고 API 호출 최소화
    """
    
    def get_sp500_symbols(self) -> List[str]:
        """S&P 500 종목 리스트 가져오기"""
        try:
            # Wikipedia에서 S&P 500 목록 가져오기 (무료!)
            url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
            tables = pd.read_html(url)
            sp500_table = tables[0]
            symbols = sp500_table['Symbol'].tolist()
            
            self.logger.info(f"S&P 500 종목 {len(symbols)}개 로드 완료")
            return symbols
            
        except Exception as e:
            self.logger.error(f"S&P 500 목록 로드 실패: {e}")
            # 백업: 주요 종목들
            return ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'META', 'NVDA', 'BRK-B', 'JNJ', 'V']
    
    
    
    def get_basic_info_batch(self, symbols: List[str]) -> pd.DataFrame:
        """
        배치로 기본 정보 가져오기 (핵심!)
        - 한 번의 API 호출로 여러 종목 정보 수집
        """
        try:
            # 최대 100개씩 배치 처리
            batch_size = 10
            all_data = []
            
            for i in range(0, len(symbols), batch_size):

                batch_symbols = symbols[i:i+batch_size]
                symbols_str = ' '.join(batch_symbols)
                
                self.logger.info(f"배치 {i//batch_size + 1}: {len(batch_symbols)}개 종목 처리 중...")
                
                # yfinance로 배치 다운로드
                tickers = yf.Tickers(symbols_str)
                
                for symbol in batch_symbols:
                    try:
                        ticker = tickers.tickers[symbol]
                        info = ticker.info
                        
                        # 핵심 정보만 추출
                        basic_data = {
                            'symbol': symbol,
                            'price': info.get('currentPrice', info.get('regularMarketPrice', 0)),
                            'volume': info.get('volume', info.get('regularMarketVolume', 0)),
                            'market_cap': info.get('marketCap', 0),
                            'sector': info.get('sector', 'Unknown'),
                            'industry': info.get('industry', 'Unknown'),
                            'pe_ratio': info.get('trailingPE', 0),
                            'fifty_two_week_low': info.get('fiftyTwoWeekLow', 0),
                            'fifty_two_week_high': info.get('fiftyTwoWeekHigh', 0),
                        }
                        all_data.append(basic_data)
                        
                    except Exception as e:
                        self.logger.warning(f"{symbol} 정보 수집 실패: {e}")
                        continue
                
                # API 제한 방지를 위한 대기
                time.sleep(1)
            
            df = pd.DataFrame(all_data)
            self.logger.info(f"총 {len(df)}개 종목 기본 정보 수집 완료")
            return df
            
        except Exception as e:
            self.logger.error(f"기본 정보 수집 실패: {e}")
            return pd.DataFrame()
    
    
    def save(self, df: pd.DataFrame, filename: str) -> None:
        """
        데이터프레임을 CSV 파일로 저장
        
        Args:
            df: 저장할 데이터프레임
            filename: 저장할 파일 이름 (예: 'basic_info.csv')
        """
        try:
            # 저장 경로 설정
            save_path = f"data/{filename}"
            
            # 디렉토리가 없으면 생성
            os.makedirs("data", exist_ok=True)
            
            # CSV로 저장
            df.to_csv(save_path, index=False)
            self.logger.info(f"✅ 데이터 저장 완료: {save_path}")
            
        except Exception as e:
            self.logger.error(f"❌ 데이터 저장 실패: {e}")
            raise
    
    def read_from_csv(self, filename: str) -> pd.DataFrame:
        """
        CSV 파일에서 데이터프레임 읽기
        
        Args:
            filename: 읽을 파일 이름 (예: 'basic_info.csv')
            
        Returns:
            pd.DataFrame: 읽어온 데이터프레임
        """
        try:
            # 파일 경로 설정
            file_path = f"data/{filename}"
            
            # 파일 존재 확인
            if not os.path.exists(file_path):
                self.logger.warning(f"⚠️ 파일이 존재하지 않음: {file_path}")
                return pd.DataFrame()
            
            # CSV 읽기
            df = pd.read_csv(file_path)
            self.logger.info(f"✅ CSV 파일 읽기 완료: {len(df)}개 종목")
            
            return df
            
        except Exception as e:
            self.logger.error(f"❌ 데이터 읽기 실패: {e}")
            return pd.DataFrame()
    

    def apply_basic_filters(self, df: pd.DataFrame, criteria: ScreeningCriteria) -> pd.DataFrame:
        """기본 필터 적용 - 대부분 종목이 여기서 걸러짐!"""
        
        initial_count = len(df)
        self.logger.info(f"1차 필터링 시작: {initial_count}개 종목 스크리닝 수행")
        
        # 가격 필터
        df = df[(df['price'] >= criteria.min_price) & (df['price'] <= criteria.max_price)]
        self.logger.info(f"가격 필터 후: {len(df)}개 ({initial_count - len(df)}개 제거)")
        
        # 거래량 필터
        df = df[df['volume'] >= criteria.min_volume]
        self.logger.info(f"거래량 필터 후: {len(df)}개")
        
        # 시가총액 필터
        df = df[df['market_cap'] >= criteria.min_market_cap]
        self.logger.info(f"시가총액 필터 후: {len(df)}개")
        
        # 섹터 필터 (선택적)
        if criteria.sectors:
            df = df[df['sector'].isin(criteria.sectors)]
            self.logger.info(f"섹터 필터 후: {len(df)}개")
        
        # 결측치 제거
        df = df.dropna(subset=['price', 'volume', 'market_cap'])
        
        self.logger.info(f"✅ 1차 필터링 완료: {initial_count}개 → {len(df)}개 (제거율: {(1-len(df)/initial_count)*100:.1f}%)")
        
        return df

In [133]:
# Create screener instance
bis = BasicInfoScreener()
filename = "basic_info.csv"

if(os.path.exists("data/" + filename)):
    print("이미존재")
    saved_infos = bis.read_from_csv(filename)
else:
    print("없음")
    symbols = bis.get_sp500_symbols()
    infos = bis.get_basic_info_batch(symbols)
    bis.save(infos, filename)  # Saves to data/basic_info.csv
    saved_infos = bis.read_from_csv(filename)

saved_infos.head()





INFO:__main__:✅ CSV 파일 읽기 완료: 503개 종목


이미존재


,symbol,price,volume,market_cap,sector,industry,last_updated,data_date
0,MMM,145.07,1680417,78073921536,Industrials,Conglomerates,2025-06-16,2025-06-17
1,AOS,64.32,1551842,9140450304,Industrials,Specialty Industrial Machinery,2025-06-16,2025-06-17
2,ABT,134.01,5066198,233155952640,Healthcare,Medical Devices,2025-06-16,2025-06-17
3,ABBV,190.86,4191502,337135108096,Healthcare,Drug Manufacturers - General,2025-06-16,2025-06-17
4,ACN,314.33,2928386,196778737664,Technology,Information Technology Services,2025-06-16,2025-06-17


In [134]:

# 스크리닝 조건 설정
criteria = ScreeningCriteria(
    min_price=20.0,           # 최소 $20
    max_price=500.0,          # 최대 $500
    min_volume=500_000,        # 최소 거래량 50만
    min_market_cap=5000_000_000, # 최소 시가총액 50억 달러
    sectors=['Technology', 'Healthcare', 'Financial Services'],  # 특정 섹터만
    bottom_breakout_pct=5.0   # 바닥 대비 5% 상승
)

first_filtered_infos = bis.apply_basic_filters(saved_infos, criteria)
first_filtered_infos.head()


INFO:__main__:1차 필터링 시작: 503개 종목 스크리닝 수행
INFO:__main__:가격 필터 후: 453개 (50개 제거)
INFO:__main__:거래량 필터 후: 433개
INFO:__main__:시가총액 필터 후: 433개
INFO:__main__:섹터 필터 후: 173개
INFO:__main__:✅ 1차 필터링 완료: 503개 → 173개 (제거율: 65.6%)


,symbol,price,volume,market_cap,sector,industry,last_updated,data_date
2,ABT,134.01,5066198,233155952640,Healthcare,Medical Devices,2025-06-16,2025-06-17
3,ABBV,190.86,4191502,337135108096,Healthcare,Drug Manufacturers - General,2025-06-16,2025-06-17
4,ACN,314.33,2928386,196778737664,Technology,Information Technology Services,2025-06-16,2025-06-17
5,ADBE,401.73,6791179,171217338368,Technology,Software - Application,2025-06-16,2025-06-17
6,AMD,126.39,100302734,204928745472,Technology,Semiconductors,2025-06-16,2025-06-17


In [135]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
from typing import List, Dict, Optional
import concurrent.futures
from dataclasses import dataclass
from datetime import timezone

class TechnicalScreener(SmartStockScreener):
    """
    기술적 분석 기반 2차 필터링
    - 1차 통과 종목들만 상세 분석
    - 바닥 돌파 등 구체적 조건 확인
    """
    
    def __init__(self):
        self.logger = self._setup_logging()
        self.history_dir = "data/history"
        os.makedirs(self.history_dir, exist_ok=True)
    
    def get_history_file_path(self, symbol: str) -> str:
        """Get the path for storing historical data"""
        return os.path.join(self.history_dir, f"{symbol}_history.csv")
    
    def save_history_data(self, symbol: str, data: pd.DataFrame) -> None:
        """Save historical data to CSV"""
        try:
            file_path = self.get_history_file_path(symbol)
            data.to_csv(file_path)
            self.logger.info(f"✅ Historical data saved for {symbol}")
        except Exception as e:
            self.logger.error(f"❌ Failed to save historical data for {symbol}: {e}")
    
    def load_history_data(self, symbol: str) -> pd.DataFrame:
        """Load historical data from CSV if exists"""
        try:
            file_path = self.get_history_file_path(symbol)
            if os.path.exists(file_path):
                data = pd.read_csv(file_path, index_col=0, parse_dates=True)
                self.logger.info(f"✅ Loaded historical data for {symbol}")
                return data
            return None
        except Exception as e:
            self.logger.error(f"❌ Failed to load historical data for {symbol}: {e}")
            return None
    
    def get_historical_data(self, symbol: str, lookback_days: int = 20) -> pd.DataFrame:
    
        """Get historical data, either from CSV or by downloading"""
        try:
            # Try to load from CSV first
            data = self.load_history_data(symbol)
            
            # Check if we have enough recent data
            end_date = datetime.now(tz=timezone.utc)
            start_date = end_date - timedelta(days=lookback_days * 2)
            start_date = start_date.replace(tzinfo=timezone.utc)
            
            if data is not None:
                if data.index[-1] >= start_date:
                    self.logger.info(f"Using cached data for {symbol}")
                    return data
            
            # If no cached data or data is too old, download new data
            self.logger.info(f"Downloading new data for {symbol}")
            
            ticker = yf.Ticker(symbol)
            data = ticker.history(start=start_date, end=end_date)
            
            if not data.empty:
                self.save_history_data(symbol, data)
            
            return data
            
        except Exception as e:
            self.logger.error(f"❌ Failed to get historical data for {symbol}: {e}")
            return pd.DataFrame()
    

    def analyze_bottom_breakout(self, symbol: str, lookback_days: int = 20) -> Optional[Dict]:
        try:
            data = self.get_historical_data(symbol, lookback_days + 1)  # +1 for today 포함

            print("================")
            print("symbol", symbol)
   

            if len(data) < lookback_days + 1:
                print("데이터 길이 불충분")
                return None
            
            # 바닥 계산 (lookback_days만큼의 과거 구간에서)
            recent_lows = data['Low'].iloc[-(lookback_days + 1):-1]  # 오늘은 제외
            bottom_price = recent_lows.min()
            bottom_date = recent_lows.idxmin()
            breakout_price = bottom_price * 1.05
            stop_loss_price = bottom_price * 0.95

            # 오늘 종가
            current_price = data['Close'].iloc[-1]

            # 어제까지 종가들이 breakout_price 미만이어야 함
            prev_closes = data['Close'].iloc[-(lookback_days): -1]
            has_broken_before = (prev_closes >= breakout_price).any()
            is_breakout_today = current_price >= breakout_price

            # 조건: 이전엔 돌파 안 했고 오늘 처음 돌파한 경우
            is_fresh_breakout = (not has_broken_before) and is_breakout_today

            avg_volume = data['Volume'].iloc[-11:-1].mean()  # 어제까지 10일 평균
            recent_volume = data['Volume'].iloc[-1]
            volume_ratio = recent_volume / avg_volume if avg_volume > 0 else 0

            result = {
                'symbol': symbol,
                'current_price': current_price,
                'bottom_date': bottom_date,
                'bottom_price': bottom_price,
                'breakout_price': breakout_price,
                'stop_loss_price': stop_loss_price,
                'is_breakout_today': is_breakout_today,
                'is_fresh_breakout': is_fresh_breakout,
                'price_from_bottom_pct': ((current_price - bottom_price) / bottom_price) * 100,
                'volume_ratio': volume_ratio,
                'avg_volume_10d': avg_volume,
                'analysis_date': datetime.now()
            }

            return result

        except Exception as e:
            self.logger.warning(f"{symbol} 기술적 분석 실패: {e}")
            return None

    def filter(self, results: List[Dict]) -> List[Dict]:
        return [r for r in results if r.get("is_fresh_breakout")]
        
    def batch_technical_analysis(self, symbols: List[str], max_workers: int = 10) -> List[Dict]:
        """병렬 처리로 기술적 분석 (속도 향상!)"""
        
        self.logger.info(f"2차 기술적 분석 시작: {len(symbols)}개 종목")
        
        results = []
        
        # 병렬 처리로 속도 향상
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            # 모든 작업 제출
            future_to_symbol = {
                executor.submit(self.analyze_bottom_breakout, symbol): symbol 
                for symbol in symbols
            }
            
            # 결과 수집
            for future in concurrent.futures.as_completed(future_to_symbol):
                result = future.result()
                if result:
                    results.append(result)
        
        self.logger.info(f"✅ 2차 분석 완료: {len(results)}개 종목 분석 성공")
        
        return results


In [136]:
second_list = first_filtered_infos['symbol'].tolist()

print("second_list", second_list)

ts = TechnicalScreener()
remaining_symbols = ts.batch_technical_analysis(second_list)
print("remaining_symbols", remaining_symbols)

#filtered_symbols = ts.filter(remaining_symbols)
technical_df = pd.DataFrame(remaining_symbols)

if not technical_df.empty:
    # 기본 정보와 기술적 분석 결과 병합
    final_df = pd.merge(first_filtered_infos, technical_df, on='symbol', how='inner')
        
    # 정렬: 바닥에서 상승률 순
    breakout_df = final_df.sort_values('price_from_bottom_pct', ascending=False)
    
else:
    breakout_df = pd.DataFrame()

INFO:__main__:2차 기술적 분석 시작: 173개 종목
INFO:__main__:✅ Loaded historical data for ABT
INFO:__main__:✅ Loaded historical data for ACN
INFO:__main__:Using cached data for ABT
INFO:__main__:✅ Loaded historical data for ABBV
INFO:__main__:✅ Loaded historical data for ADBE
INFO:__main__:Using cached data for ACN
INFO:__main__:✅ Loaded historical data for A
INFO:__main__:✅ Loaded historical data for AFL
INFO:__main__:✅ Loaded historical data for AMD
INFO:__main__:✅ Loaded historical data for ALGN
INFO:__main__:✅ Loaded historical data for AKAM
INFO:__main__:Using cached data for ABBV
INFO:__main__:✅ Loaded historical data for ALL
INFO:__main__:✅ Loaded historical data for AXP
INFO:__main__:Using cached data for ADBE
INFO:__main__:Using cached data for A
INFO:__main__:Using cached data for AFL
INFO:__main__:✅ Loaded historical data for AIG
INFO:__main__:Using cached data for AMD
INFO:__main__:Using cached data for ALGN
INFO:__main__:Using cached data for AKAM
INFO:__main__:Using cached data for 

second_list ['ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AFL', 'A', 'AKAM', 'ALGN', 'ALL', 'AXP', 'AIG', 'AMGN', 'APH', 'ADI', 'ANSS', 'AON', 'APO', 'AAPL', 'AMAT', 'ACGL', 'ANET', 'AJG', 'ADSK', 'ADP', 'BAC', 'BAX', 'BDX', 'BRK.B', 'TECH', 'BIIB', 'BX', 'BK', 'BSX', 'BMY', 'AVGO', 'BRO', 'CDNS', 'COF', 'CAH', 'CBOE', 'CDW', 'COR', 'CNC', 'CRL', 'SCHW', 'CB', 'CI', 'CINF', 'CSCO', 'C', 'CFG', 'CME', 'CTSH', 'COIN', 'COO', 'GLW', 'CPAY', 'CRWD', 'CVS', 'DHR', 'DVA', 'DAY', 'DELL', 'DXCM', 'EW', 'ELV', 'ENPH', 'EPAM', 'FIS', 'FITB', 'FSLR', 'FI', 'FTNT', 'FTV', 'BEN', 'GRMN', 'IT', 'GEHC', 'GEN', 'GILD', 'GL', 'GDDY', 'HIG', 'HCA', 'HSIC', 'HOLX', 'HPQ', 'HUM', 'IBM', 'INCY', 'PODD', 'INTC', 'ICE', 'IQV', 'JBL', 'JKHY', 'JNJ', 'JPM', 'JNPR', 'KEYS', 'KKR', 'LH', 'LRCX', 'LDOS', 'L', 'MTB', 'MMC', 'MDT', 'MRK', 'MET', 'MCHP', 'MU', 'MSFT', 'MRNA', 'MOH', 'MCO', 'MS', 'MSI', 'NDAQ', 'NTAP', 'NTRS', 'NVDA', 'NXPI', 'ON', 'ORCL', 'PLTR', 'PANW', 'PAYX', 'PYPL', 'PFE', 'PNC', 'PFG', 'PGR', 'PRU', '

INFO:__main__:✅ Loaded historical data for CME
INFO:__main__:Using cached data for CSCO
INFO:__main__:✅ Loaded historical data for CTSH
INFO:__main__:Using cached data for C
INFO:__main__:✅ Loaded historical data for COIN
INFO:__main__:✅ Loaded historical data for COO
INFO:__main__:Using cached data for CFG
INFO:__main__:✅ Loaded historical data for GLW
INFO:__main__:Using cached data for CME
INFO:__main__:✅ Loaded historical data for CPAY
INFO:__main__:✅ Loaded historical data for CRWD
INFO:__main__:Using cached data for CTSH
INFO:__main__:Using cached data for COIN
INFO:__main__:Using cached data for COO
INFO:__main__:✅ Loaded historical data for CVS
INFO:__main__:✅ Loaded historical data for DHR
INFO:__main__:Using cached data for GLW
INFO:__main__:✅ Loaded historical data for DVA
INFO:__main__:Using cached data for CPAY
INFO:__main__:Using cached data for CRWD
INFO:__main__:✅ Loaded historical data for DAY
INFO:__main__:✅ Loaded historical data for DELL
INFO:__main__:✅ Loaded histo

symbol CSCO
symbol C
symbol CFG
symbol CME
symbol CTSH
symbol COIN
symbol COO
symbol GLW
symbol CPAY
symbol CRWD
symbol CVS
symbol DHR
symbol DVA
symbol DAY
symbol DELL
symbol DXCM
symbol EW
symbol ELV


INFO:__main__:Using cached data for ENPH
INFO:__main__:✅ Loaded historical data for FITB
INFO:__main__:✅ Loaded historical data for FSLR
INFO:__main__:Using cached data for EPAM
INFO:__main__:✅ Loaded historical data for FI
INFO:__main__:✅ Loaded historical data for FTNT
INFO:__main__:Using cached data for FIS
INFO:__main__:✅ Loaded historical data for FTV
INFO:__main__:✅ Loaded historical data for BEN


...

symbol GL
symbol GDDY
symbol HIG
symbol HCA
symbol HSIC
symbol HOLX
symbol HPQ
symbol HUM
symbol IBM
symbol INCY
symbol PODD
symbol INTC
symbol ICE
symbol IQV
symbol JBL


INFO:__main__:Using cached data for JKHY
INFO:__main__:Using cached data for JNJ
INFO:__main__:✅ Loaded historical data for KKR
INFO:__main__:Using cached data for JPM
INFO:__main__:✅ Loaded historical data for LH
INFO:__main__:Using cached data for JNPR
INFO:__main__:✅ Loaded historical data for LRCX
INFO:__main__:✅ Loaded historical data for LDOS
INFO:__main__:Using cached data for KEYS
INFO:__main__:✅ Loaded historical data for L
INFO:__main__:Using cached data for KKR
INFO:__main__:✅ Loaded historical data for MTB
INFO:__main__:✅ Loaded historical data for MMC
INFO:__main__:Using cached data for LH
INFO:__main__:✅ Loaded historical data for MDT
INFO:__main__:Using cached data for LRCX
INFO:__main__:Using cached data for LDOS
INFO:__main__:✅ Loaded historical data for MRK
INFO:__main__:Using cached data for L
INFO:__main__:✅ Loaded historical data for MET
INFO:__main__:Using cached data for MTB
INFO:__main__:Using cached data for MMC
INFO:__main__:✅ Loaded historical data for MCHP
I

symbol JKHY
symbol JNJ
symbol JPM
symbol JNPR
symbol KEYS
symbol KKR
symbol LH
symbol LRCX
symbol LDOS
symbol L
symbol MTB
symbol MMC
symbol MDT
symbol MRK
symbol MET
symbol MCHP
symbol MU
symbol MSFT
symbol MRNA
symbol MOH
symbol MCO
symbol MS
symbol MSI
symbol NDAQ
symbol NTAP
symbol NTRS
symbol NVDA
symbol NXPI
symbol ON
symbol ORCL
symbol PLTR
symbol PANW
symbol PAYX
symbol PYPL
symbol PNC
symbol PFE
symbol PFG
symbol PGR
symbol PRU
symbol PTC
symbol QCOM
symbol DGX
symbol RJF
symbol RF
symbol RMD
symbol RVTY
symbol CRM
symbol STX
symbol SWKS
symbol SOLV
symbol STT
symbol SYK
symbol SMCI
symbol SYF
symbol SNPS


INFO:__main__:✅ Loaded historical data for TRV
INFO:__main__:Using cached data for TEL
INFO:__main__:Using cached data for TROW
INFO:__main__:✅ Loaded historical data for TRMB
INFO:__main__:Using cached data for TER
INFO:__main__:✅ Loaded historical data for TFC
INFO:__main__:✅ Loaded historical data for USB
INFO:__main__:Using cached data for TXN
INFO:__main__:Using cached data for TMO
INFO:__main__:✅ Loaded historical data for UBER
INFO:__main__:Using cached data for TRV
INFO:__main__:Using cached data for TRMB
INFO:__main__:✅ Loaded historical data for UNH
INFO:__main__:✅ Loaded historical data for UHS
INFO:__main__:Using cached data for TFC
INFO:__main__:✅ Loaded historical data for VRTX
INFO:__main__:Using cached data for USB
INFO:__main__:Using cached data for UBER
INFO:__main__:✅ Loaded historical data for V
INFO:__main__:✅ Loaded historical data for WRB
INFO:__main__:✅ Loaded historical data for WFC
INFO:__main__:✅ Loaded historical data for WST
INFO:__main__:Using cached data 

symbol TEL
symbol TROW
symbol TER
symbol TXN
symbol TMO
symbol TRV
symbol TRMB
symbol TFC
symbol USB
symbol UBER
symbol UNH
symbol UHS
symbol VRTX
symbol V
symbol WRB
symbol WFC
symbol WST
symbol WDC
symbol WTW
symbol WDAY
symbol ZBH
symbol ZTS
remaining_symbols [{'symbol': 'ABT', 'current_price': 135.6199951171875, 'bottom_date': Timestamp('2025-05-15 00:00:00-0400', tz='UTC-04:00'), 'bottom_price': 128.85000610351562, 'breakout_price': 135.29250640869142, 'stop_loss_price': 122.40750579833984, 'is_breakout_today': True, 'is_fresh_breakout': False, 'price_from_bottom_pct': 5.254162741934987, 'volume_ratio': 0.6827767250221658, 'avg_volume_10d': 5752100.0, 'analysis_date': datetime.datetime(2025, 6, 18, 11, 12, 47, 821599)}, {'symbol': 'ACN', 'current_price': 311.7099914550781, 'bottom_date': Timestamp('2025-05-23 00:00:00-0400', tz='UTC-04:00'), 'bottom_price': 307.8699951171875, 'breakout_price': 323.2634948730469, 'stop_loss_price': 292.4764953613281, 'is_breakout_today': False, 'is

In [137]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests
import time
from typing import List, Dict, Optional
import concurrent.futures
from dataclasses import dataclass
import json


class ExternalScreener(SmartStockScreener):
    """
    외부 스크리너 활용 - 이미 필터링된 결과 사용
    - Finviz, Yahoo Screener 등 활용
    - 가장 효율적인 방법!
    """
    
    def get_finviz_screener_results(self, custom_filters: List[str] = None) -> List[str]:
        """
        Finviz 스크리너 결과 활용
        - 이미 필터링된 종목들만 가져옴
        - 데이터 다운로드 최소화!
        """
        try:
            # finvizfinance 라이브러리 필요: pip install finvizfinance
            from finvizfinance.screener.overview import Overview
            
            foverview = Overview()
            
            # 기본 필터: 거래량, 가격 등
            default_filters = [
                'sh_avgvol_o200',    # 평균 거래량 20만 이상
                'sh_price_o5',       # 가격 $5 이상
                'cap_midover',       # 중형주 이상
            ]
            
            filters = custom_filters if custom_filters else default_filters
            foverview.set_filter(filters_dict=dict(zip(filters, [None]*len(filters))))
            
            # 스크리너 실행
            df = foverview.screener_view()
            symbols = df['Ticker'].tolist()
            
            self.logger.info(f"Finviz에서 {len(symbols)}개 종목 발견")
            return symbols
            
        except ImportError:
            self.logger.warning("finvizfinance 라이브러리 없음. pip install finvizfinance")
            return []
        except Exception as e:
            self.logger.error(f"Finviz 스크리너 실행 실패: {e}")
            return []
    
    def get_yahoo_screener_results(self) -> List[str]:
        """Yahoo Finance 스크리너 결과 활용"""
        try:
            # Yahoo Finance 스크리너 URL (예시)
            # 실제로는 requests + BeautifulSoup로 파싱 필요
            
            # 간단한 예시: 52주 신고가 근처 종목들
            symbols = ['AAPL', 'MSFT', 'NVDA', 'GOOGL', 'AMZN']  # 예시
            
            self.logger.info(f"Yahoo 스크리너에서 {len(symbols)}개 종목 발견")
            return symbols
            
        except Exception as e:
            self.logger.error(f"Yahoo 스크리너 실행 실패: {e}")
            return []


In [138]:
es =ExternalScreener()
es.get_finviz_screener_results()
es.get_finviz_screener_results()

ERROR:__main__:Finviz 스크리너 실행 실패: Invalid filter 'sh_avgvol_o200'. Possible filter: ['Exchange', 'Index', 'Sector', 'Industry', 'Country', 'Market Cap.', 'P/E', 'Forward P/E', 'PEG', 'P/S', 'P/B', 'Price/Cash', 'Price/Free Cash Flow', 'EPS growththis year', 'EPS growthnext year', 'EPS growthpast 5 years', 'EPS growthnext 5 years', 'Sales growthpast 5 years', 'EPS growthqtr over qtr', 'Sales growthqtr over qtr', 'Dividend Yield', 'Return on Assets', 'Return on Equity', 'Return on Investment', 'Current Ratio', 'Quick Ratio', 'LT Debt/Equity', 'Debt/Equity', 'Gross Margin', 'Operating Margin', 'Net Profit Margin', 'Payout Ratio', 'InsiderOwnership', 'InsiderTransactions', 'InstitutionalOwnership', 'InstitutionalTransactions', 'Float Short', 'Analyst Recom.', 'Option/Short', 'Earnings Date', 'Performance', 'Performance 2', 'Volatility', 'RSI (14)', 'Gap', '20-Day Simple Moving Average', '50-Day Simple Moving Average', '200-Day Simple Moving Average', 'Change', 'Change from Open', '20-Day H

[]

In [139]:
breakout_df

""


In [140]:
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf
import os

def visualize_breakout_analysis(symbol: str, breakout_df: pd.DataFrame, lookback_days: int = 20):
    """
    Visualize breakout analysis for a given stock using pre-calculated breakout data
    
    Args:
        symbol: Stock symbol
        breakout_df: DataFrame containing breakout analysis results
        lookback_days: Number of days to look back for analysis
    """
    # Get historical data
    history_dir = "data/history"
    file_path = os.path.join(history_dir, f"{symbol}_history.csv")
    
    if os.path.exists(file_path):
        data = pd.read_csv(file_path, index_col=0, parse_dates=True)
    else:
        end_date = datetime.now(tz=timezone.utc)
        start_date = end_date - timedelta(days=lookback_days * 2)
        ticker = yf.Ticker(symbol)
        data = ticker.history(start=start_date, end=end_date)
    
    if data.empty:
        print(f"No data available for {symbol}")
        return
    
    # Get breakout data from breakout_df
    symbol_data = breakout_df[breakout_df['symbol'] == symbol].iloc[0]
    bottom_price = symbol_data['bottom_price']
    breakout_price = symbol_data['breakout_price']
    stop_loss_price = symbol_data['stop_loss_price']
    bottom_date = pd.to_datetime(symbol_data['bottom_date'])
    
    # Create the plot
    plt.figure(figsize=(15, 8))
    sns.set_style("whitegrid")
    
    # Plot price action
    plt.plot(data.index, data['Close'], label='Close Price', color='blue', linewidth=2)
    
    # Plot bottom and breakout levels
    plt.axhline(y=bottom_price, color='red', linestyle='--', label='Bottom Price')
    plt.axhline(y=breakout_price, color='green', linestyle='--', label='Breakout Level (5%)')
    plt.axhline(y=stop_loss_price, color='blue', linestyle='--', label='Stop Loss Price')
    
    # Mark the bottom point
    plt.scatter(bottom_date, bottom_price, color='red', s=100, zorder=5, label='Bottom Point')
    
    # Add volume bars at the bottom
    ax2 = plt.twinx()
    ax2.bar(data.index, data['Volume'], alpha=0.3, color='gray', label='Volume')
    
    # Customize the plot
    plt.title(f'{symbol} Breakout Analysis', fontsize=15, pad=15)
    plt.xlabel('Date', fontsize=12)
    plt.ylabel('Price ($)', fontsize=12)
    ax2.set_ylabel('Volume', fontsize=12)
    
    # Add legend
    lines1, labels1 = plt.gca().get_legend_handles_labels()
    lines2, labels2 = ax2.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    # Show current price and breakout status
    current_price = data['Close'].iloc[-1]
    breakout_status = "BREAKOUT" if current_price >= breakout_price else "NO BREAKOUT"
    price_from_bottom_pct = ((current_price - bottom_price) / bottom_price) * 100
    
    info_text = (
        f'Current Price: ${current_price:.2f}\n'
        f'Bottom Price: ${bottom_price:.2f}\n'
        f'Bottom Date: {bottom_date.strftime("%Y-%m-%d")}\n'
        f'Price from Bottom: {price_from_bottom_pct:.1f}%\n'
        f'Breakout Status: {breakout_status}'
    )
    
    plt.text(0.02, 0.98, info_text,
             transform=plt.gca().transAxes, fontsize=12,
             bbox=dict(facecolor='white', alpha=0.8))
    
    plt.tight_layout()
    plt.show()

def visualize_all_breakouts(breakout_df: pd.DataFrame):
    """
    Visualize breakout analysis for all symbols in the breakout DataFrame
    
    Args:
        breakout_df: DataFrame containing breakout analysis results
    """
    for _, row in breakout_df.iterrows():
        symbol = row['symbol']
        print(f"\nAnalyzing {symbol}...")
        visualize_breakout_analysis(symbol, breakout_df)
        print("\n" + "="*50 + "\n") 

In [141]:
visualize_all_breakouts(breakout_df)